Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

Data 

In [ ]:
data_path = 'Datasets/TRANSCHEM.BO.csv'
data = pd.read_csv(data_path)
del data['Date']
data.head()

Z-Score Normalization of Dataset

In [ ]:
scaler = StandardScaler()
trans_data = scaler.fit_transform(np.array(data))

Params

In [ ]:
seq_len = 10
train_divide = 0.8
val_divide = 0.1
test_divide = 0.1

Dataset Generation

In [ ]:
x = []
y = []
for i in range(len(trans_data)-seq_len):
    x.append(trans_data[i:i+seq_len, :])
    y.append(trans_data[i+seq_len, 4])
x = np.array(x)
y = np.array(y)


Data Split

In [ ]:
train_ind = int(len(trans_data)*train_divide)
val_ind = train_ind + int(len(trans_data)*val_divide)
x_train = x[:train_ind]
y_train = y[:train_ind]
x_val = x[train_ind:val_ind]
y_val = y[train_ind:val_ind]
x_test = x[val_ind:]
y_test = y[val_ind:]
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

Model 


In [ ]:
model = keras.models.Sequential()
model.add(layers.Conv1D(32, 1, padding='same',
          activation='tanh', input_shape=(10, 6)))
model.add(layers.MaxPool1D(pool_size=1, padding='same'))
model.add(layers.LSTM(64, activation='tanh'))
model.add(layers.Dense(1))

Compile Model

In [ ]:
model.compile(loss='mean_absolute_error',optimizer=keras.optimizers.Adam(learning_rate=0.001))
model.summary()

Architecture

In [ ]:
keras.utils.plot_model(
    model, "Model_Architecture.png", show_shapes=True)


Training

In [ ]:
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=100,batch_size=64)

Testing

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)


Plot of Time Series Prediction

In [ ]:
mean_ = scaler.mean_[4]
sd_ = np.sqrt(scaler.var_[4])
trans_predicted_close = model(x_test)
prediction_close = sd_*trans_predicted_close + mean_
actual_close = sd_*y_test + mean_

Plots

In [ ]:
plt.plot(actual_close, color='red')
plt.plot(prediction_close, color='blue')
plt.xlabel('Time')
plt.ylabel('Closing Stock Prices')
plt.title("Time Series Prediction")
plt.legend(['Real Price', 'Predicted Price'])
plt.savefig('./Results/'+data_path.split('/')[1].split('.')[0])